In [1]:
# The necessary installation(s) 

#!pip install numpy==1.21.4
#!pip install pandas==1.3.4
#!pip install keras==2.1.6
!pip install scikit-learn


# IMPORTANT: 

# NEVER RUN TENSORFLOW / KERAS FROM OUR OWN SYSTEM. 
# ITS INADEQUATE FOR SUCH LIBRARIES AND EVERYTHING CRASHES! 
# WE GET COMPLETELY RANDOM ERROR MESSGAES (WHICH OTHERWISE NEVER HAPPENED), 
# AND THEY CAN'T BE CORRECTED AT ALL. 
# THESE ARE DUE TO OUR LIMITED SPACE / ABILITY OF OUR PERSONAL SYSTEM(S) 
# TO HANDLE SUCH LIBRARIES. 

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
#import sklearn.metrics as metrics

import warnings
warnings.simplefilter('ignore', FutureWarning)

In [25]:
# Loading the data 

concrete_data = pd.read_csv("concrete_data.csv")
print("Dimensions (rows & columns) of the dataset:", concrete_data.shape)
concrete_data.head()

Dimensions (rows & columns) of the dataset: (1030, 9)


,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [26]:
# Checking the presence of missing data
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [27]:
# Selecting the predictors 

predictors = concrete_data[concrete_data.columns[concrete_data.columns!='Strength']]
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [28]:
# Selecting the target 

target = concrete_data['Strength']
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [29]:
# Normalizing the predictors 

predictors_norm = (predictors - predictors.mean()) / predictors.std() 
predictors_norm.head() 

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [30]:
# Saving the #inputs required (same as the #predictors) to be used in the regression model 

n_cols = predictors_norm.shape[1]
n_cols

8

In [31]:
# Spliting the training and testing set using the scikit-learn function 

x_train, x_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3)
print ('Train set:', x_train.shape,  y_train.shape)
print ('Test set:', x_test.shape,  y_test.shape)

Train set: (721, 8) (721,)
Test set: (309, 8) (309,)


In [10]:
import keras

Using TensorFlow backend.


In [32]:
from keras.models import Sequential
from keras.layers import Dense

In [33]:
# The function for regression model with one hidden layer of 10 nodes

def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [34]:
# Function for the Mean Squre Error

def MeanSqureError_Estimator():
    model = regression_model()
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=50, verbose=0)
    model.evaluate(x_test, y_test, verbose=0)
    y_pred = model.predict(x_test, verbose=0) 
    return mean_squared_error(y_test, y_pred)

# Note: The default value of "batch_size" is 32; hence #batches (the small chuncks to which the full 
# dataset has been split) is (sample size of the training set)/32 = 721/32, which lies b/w 22 & 23). 
# That's what is shown as 23/23 below (if we set "verbose" as 1 or 2). 

In [35]:
results = []

for i in range(1, 51):
    results.append({"Iteration": i, "MSE": MeanSqureError_Estimator()})

results_df = pd.DataFrame(results)
print(results_df)

    Iteration         MSE
0           1  523.930825
1           2  485.588846
2           3  385.673191
3           4  422.256988
4           5  557.803372
5           6  366.816454
6           7  306.826385
7           8  305.591141
8           9  536.899255
9          10  363.776211
10         11  455.104872
11         12  738.740644
12         13  361.214682
13         14  324.552436
14         15  661.047862
15         16  287.506761
16         17  286.032364
17         18  318.740542
18         19  351.430043
19         20  330.955549
20         21  261.891950
21         22  331.967843
22         23  392.911862
23         24  719.965976
24         25  309.864170
25         26  274.233695
26         27  381.711129
27         28  711.246231
28         29  368.363063
29         30  337.034886
30         31  473.354020
31         32  303.986443
32         33  304.346562
33         34  293.325716
34         35  449.409960
35         36  344.561839
36         37  309.428030
37         3

In [36]:
# Mean and Standard deviation 

mean_mse = results_df['MSE'].mean()
print("Mean value of all the mean square errors", mean_mse)

sd_mse = results_df['MSE'].std() 
print("Standard devition of all the mean square errors", sd_mse)

Mean value of all the mean square errors 388.0332814678321
Standard devition of all the mean square errors 120.82860871422989


In [37]:
# Q: How does the mean of the mean squared errors compare to that from Step A? 

# Ans: The mean squared errors has reduced after normalization. 

In [ ]:
# SOME ADDITIONAL ASPECTS I TRIED: 

# Alternate method:
y_pred = np.array(y_pred)
y_test = np.array(y_test)

MSE = np.mean((y_pred - y_test) ** 2)
MSE 
# NOTE: 
# This result should ideally match the one calculated using the scikit-learn function "mean_squared_error". 
# The mismatch arises because the dataset(s) may not be 1-dimensional, which is corrected below. 

In [ ]:
# Flattening the datasets to get the correct result (same as the scikit-learn function) 

y_test = y_test.ravel()
#print("Testing set:", y_test)

y_pred = y_pred.ravel()
#print("Predictions:", y_pred)

np.mean((y_pred - y_test) ** 2)